# Bias Visualization

### Dependencies
Note that we use the scikit-learn confusion matrix utility to help calculate some of the fairness metrics. We generate the visualizations using plotly Express.

In [27]:
%%capture
%pip install kaleido numpy pandas plotly scikit-learn tqdm ipywidgets

In [28]:
import os
from math import sqrt
from typing import Any, Dict, List

import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import clear_output, display
from sklearn.metrics import confusion_matrix as ConfusionMatrix
from tqdm.auto import tqdm

### Global Constants

This notebook will visualize how each of the following variable affects the fairness towards a particular protected group.

In [29]:
INDEPENDENT_VARIABLES = ["model", "dataset", "num_params"]

## Utilities

### Plotting Utils

In [30]:
def format_metric_name(metric_name: str) -> str:
    """
    Prettify diagram texts by replacing
    snake case with regular title case.
    """
    output_words = []
    for word in metric_name.split("_"):
        word = word.title() if word not in ["FPR", "TPR"] else word.upper()
        output_words.append(word)

    return " ".join(output_words)


# Modify the names of the models and the datasets for display purposes
def modify_model_name(model_name: str) -> str:
    model_name = model_name.replace("facebook/", "")
    model_name = model_name.replace("data/models/", "")
    model_name = model_name.replace("model-weights/", "")
    
    if model_name.startswith("opt"):
        return model_name.replace("opt", "OPT").upper()
    elif model_name.startswith("roberta"):
        return model_name.replace("roberta", "RoBERTa")
    elif model_name == "/llama/7B":
        return "LLaMA-7B"
    elif model_name == "/llama/13B":
        return "LLaMA-13B"
    elif model_name == "/Llama-2-7b":
        return "Llama2-7B"
    elif model_name == "/Llama-2-13b":
        return "Llama2-13B"
    else:
        return model_name


def modify_dataset_name(dataset_name: str) -> str:
    if "SST5" in dataset_name:
        return "SST5 Grouped 9-Shot Prompt"
    elif "SemEval" in dataset_name:
        return "SemEval 9-shot Prompt"
    elif "ZeroShot" in dataset_name:
        return "Zero-shot Prompt"
    return dataset_name

## Getting Predictions Ready

### Load Prediction Files
Load all prediction `tsv` files from a given folder into a dataframe.

In [31]:
prediction_folder: str = "/Users/david/Documents/ResearchPapers/SoftPromptTuningFairness/PurePromptingPredictions/pooled_preds/"
prediction_files: List[str] = os.listdir(prediction_folder)
prediction_tsv_paths: List[str] = [
    os.path.join(prediction_folder, prediction_file)
    for prediction_file in prediction_files
    if prediction_file.endswith(".tsv") and "placeholder" not in prediction_file
]

output_folder: str = "stats"
output_table = pd.DataFrame()

for prediction_tsv_path in tqdm(prediction_tsv_paths, ncols=80):
    prediction_tsv_filename = os.path.basename(prediction_tsv_path)
    dataframe = pd.read_csv(prediction_tsv_path, delimiter="\t")
    del dataframe["text"]
    

    if output_table is None:
        output_table = dataframe
    else:
        assert isinstance(output_table, pd.DataFrame)
        output_table = pd.concat([output_table, dataframe])

  0%|                                                    | 0/45 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Calculate Statistics
Since there can be multiple examples for each protected group, we apply the `groupby` method to compute aggregated statistics across all the examples of each protected group.

Refer to the following function for details on how we implemented the metrics.

### Function for aggregating statistics


Below you will find a function that takes in a dataframe. This dataframe is will be automatically generated using the `pd.DataFrame.groupby`. The content of this dataframe is a subset of one particular prediction tsv for a particular protected group (e.g., the "young" group for the protected class "age"). 

The function will return a `pd.Series` (like a dictionary) mapping the following fairness metrics to their floating point values:
- Accuracy
- TPR(Positive)
- TPR(Neutral)
- TPR(Negative)
- FPR(Positive)
- FPR(Neutral)
- FPR(Negative)

Also refer to https://stackoverflow.com/a/50671617 for details on how the TPR and FPR values are computed.

In [ ]:
def get_stats(dataframe: pd.DataFrame) -> pd.Series:
    """
    Input: dataframe representing all predictions for a particular
    protected group in a particular run.
    """
    output: Dict[str, float] = {}

    num_examples = len(dataframe)
    num_correct = np.sum(dataframe["y_pred"] == dataframe["y_true"])

    output["accuracy"] = num_correct / num_examples

    # See https://stackoverflow.com/a/50671617
    confusion_matrix = ConfusionMatrix(dataframe["y_true"], dataframe["y_pred"])

    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.sum() - (FP + FN + TP)

    FPR = FP / (FP + TN)
    FNR = FN / (TP + FN)
    TPR = 1 - FNR

    for label_index, label in enumerate(["negative", "neutral", "positive"]):
        output[label + "_FPR"] = FPR[label_index]
        output[label + "_TPR"] = TPR[label_index]

    return pd.Series(output)

### Calculating Per-Group Statistics

In [ ]:
# output_table
analyzed_dataframe = output_table.groupby([*INDEPENDENT_VARIABLES, "run_id", "category", "group"]).apply(get_stats)
analyzed_dataframe.query('category == "grouped_race"')[["positive_TPR"]]

positive_TPR
model    dataset  num_params run_id category     group                         
LLaMA-7B SST5     7.0        run_1  grouped_race african_american      0.775000
                                                 american_indian       0.816667
                                                 asian                 0.900000
                                                 hispanic              0.925000
                                                 pacific_islander      0.975000
...                                                                         ...
OPT-6.7B ZeroShot 6.7        run_5  grouped_race american_indian       0.400000
                                                 asian                 0.550000
                                                 hispanic              0.375000
                                                 pacific_islander      0.575000
                                                 white                 0.150000

[270 rows x 1 columns]

In [ ]:
output_table = analyzed_dataframe.reset_index()
os.makedirs(output_folder, exist_ok=True)
output_table.to_csv(os.path.join(output_folder, "aggregated.csv"))

## Calculating Statistics

Before we begin, note that some of the protected classes include a large number of groups. You may filter on the protected groups to include in each category. If a category isn't in this dictionary, the visualizations will include all the groups under that category by default.

In [ ]:
SELECTED_GROUPS = {
    "grouped_religion": ["christianity", "islam", "judaism"],
    "grouped_disability": [
        "without",
        "cognitive",
        "physical",
        "hearing",
        "sight",
        "chronic_illness",
        "mobility",
        "mental_health",
    ],
}

groups_to_include = []
for category in output_table["category"].unique():
    groups = SELECTED_GROUPS.get(category)
    if groups is None:
        groups = list(set(output_table.query(f'category == "{category}"')["group"]))

    groups_to_include.extend(groups)

print("Groups selected:", ", ".join(groups_to_include))
output_table = output_table.query("group == @groups_to_include")

Groups selected: adult, young, old, 4q, 5q, 2q, 3q, 1q, 6q, without, cognitive, physical, hearing, sight, chronic_illness, mobility, mental_health, aab, many-genders, female, non-binary, trans, no-gender, cis, male, hispanic, american_indian, asian, pacific_islander, white, african_american, christianity, islam, judaism, other, homosexual, asexual, heterosexual, bisexual, female, male


### "Gap": How each group deviates from the category mean
Below, we will calculate how far each group deviates from the mean of that category. For example, how far does the accuracy on examples mentioning the "young" group deviate from the mean of all examples of the "age" category?

If there are more than one runs for each model, we would calculate gap metrics for each run separately. 

We use a for loop to add the following metrics of interest to our list:
- Accuracy
- False Positive Rates
    - FPR("Negative")
    - FPR("Neutral")
    - FPR("Positive")
- True Positive Rates
    - TPR("Negative")
    - TPR("Neutral")
    - TPR("Positive")

In [ ]:
metrics = ["accuracy"]
for label in ["negative", "neutral", "positive"]:
    metrics.append(label + "_FPR")
    metrics.append(label + "_TPR")

gap_metrics = []
for metric in metrics:
    gap_metrics.append(f"{metric}_gap")

print(gap_metrics)

['accuracy_gap', 'negative_FPR_gap', 'negative_TPR_gap', 'neutral_FPR_gap', 'neutral_TPR_gap', 'positive_FPR_gap', 'positive_TPR_gap']


### Confidence Interval Calculations

In the previous step, we've found the gaps for each (metric, category, group, model, run/seed). If we have more than one runs/seeds for each model, we can aggregate the results to find a confidence interval for each gap variable.

To do so, we will aggregate the previous table (metric, category, group, model, run/seed) along the run/seed axis. The result will be a table with indices (metric, category, group, model).

In [ ]:
# Create a (metric, group) placeholder table for storing the output values.
# Unlike in the previous step, "run_id" isn't part of groupby,
# and this axis will be squeezed in the result.
stats_table = (
    output_table[[*INDEPENDENT_VARIABLES, "category", "group", "accuracy"]]
    .groupby([*INDEPENDENT_VARIABLES, "category", "group"])
    .agg(["mean"])
)

In [ ]:
# Calculate gap for each metric and
# save the results in a column named f"{metrics}_gap".
for metric, gap_metric in zip(metrics, gap_metrics):
    # Calculate mean for each metric, group, and run/seed.
    group_by = [*INDEPENDENT_VARIABLES, "category", "run_id"]
    grouped = output_table[[*INDEPENDENT_VARIABLES, "category", "run_id", metric]].groupby(group_by)
    group_mean = grouped.transform("mean")

    # Broadcast the mean values across the table
    # to find the gap for each entry.
    normalized_values = output_table[metric] - group_mean[metric]
    output_table[gap_metric] = normalized_values

    # See https://stackoverflow.com/a/53522680
    # Calculate mean and stdev for each (metric, category, group, model).
    stats = (
        output_table[[*INDEPENDENT_VARIABLES, "category", "group", gap_metric]]
        .groupby([*INDEPENDENT_VARIABLES, "category", "group"])
        .agg(["mean", "count", "std"])
    )

    mean_values = []
    ci_width = []
    lower_values = []
    upper_values = []
    hypothesis_selected = []
    z_score = 1.96

    # Calculate 95% Confidence interval for each
    # (metric, category, group, model).
    for index in stats.index:
        mean, n, stdev = stats.loc[index]  # type: ignore
        lower = mean - z_score * stdev / sqrt(n)
        upper = mean + z_score * stdev / sqrt(n)

        mean_values.append(mean)
        ci_width.append(z_score * stdev / sqrt(n))
        lower_values.append(lower)
        upper_values.append(upper)

        if lower > 0:
            hypothesis_selected.append(1)
        elif upper < 0:
            hypothesis_selected.append(-1)
        else:
            hypothesis_selected.append(0)

    stats_table[gap_metric] = mean_values
    stats_table[gap_metric + "_width"] = ci_width
    stats_table[gap_metric + "_lower"] = lower_values
    stats_table[gap_metric + "_upper"] = upper_values

stats_table = stats_table.reset_index()

In [ ]:
stats_table["model"] = stats_table["model"].apply(modify_model_name)
stats_table["dataset"] = stats_table["dataset"].apply(modify_dataset_name)

## Plotting

### Compare between Groups of each category 
E.g. all groups in the "age" category

In [ ]:
datasets = ["(all)", *stats_table["dataset"].unique()]
models = ["(all)", *(stats_table["model"].unique())]
categories = stats_table["category"].unique()

##### Interactive Visualization Utilities

In [ ]:
model_selector = widgets.Dropdown(options=models, value=models[0], description="Model:")
category_selector = widgets.Dropdown(options=categories, value=categories[0], description="Category:")
metric_selector = widgets.Dropdown(options=metrics, value=metrics[0], description="Metric:")
dataset_selector = widgets.Dropdown(options=datasets, value=datasets[0], description="Dataset:")

In [ ]:
def visualize_fn(button: Any) -> None:
    """
    Function to be called when the button is pressed.
    This function clears previous output and any widget
    that has already been rendered. To add these widgets
    back (including the button) the "button" parameter
    needs to be supplied from the button action.
    """
    clear_output()

    display(model_selector)
    display(category_selector)
    display(metric_selector)
    display(dataset_selector)
    display(button)

    model = model_selector.value
    category: str = category_selector.value  # type: ignore
    metric: str = metric_selector.value  # type: ignore
    dataset: str = dataset_selector.value  # type: ignore

    metric += "_gap"

    if model == "(all)" and dataset == "(all)":
        filtered_table = stats_table[(stats_table["category"] == category)]
    elif model == "(all)":
        filtered_table = stats_table[(stats_table["category"] == category) & (stats_table["dataset"] == dataset)]
    elif dataset == "(all)":
        filtered_table = stats_table[(stats_table["category"] == category) & (stats_table["model"] == model)]
    else:
        filtered_table = stats_table[
            (stats_table["category"] == category)
            & (stats_table["model"] == model)
            & (stats_table["dataset"] == dataset)
        ]

    if len(filtered_table) == 0:
        print("None of the runs in the table matched the selections.")
        return

    # Removing the "grouped" from grouped_{category_name}
    category_name = category.split("_")[-1]
    title = f"Gap for {category_name.title()}<br>{dataset}"

    # Generate scatterplot with confidence interval.
    fig = px.scatter(
        filtered_table,
        x="group",
        facet_col="group",
        y=metric,
        color="model",
        error_y=metric + "_width",
        labels={
            "group": "Protected Group",
            metric: format_metric_name(metric),
            "model": "LM",
        },
        category_orders={
            "model": ["OPT-125M", "OPT-350M", "OPT-1.3B", "OPT-2.7B", "OPT-6.7B", "OPT-13B", "LLaMA-7B", "LLaMA-13B"]
        },
        title=title,
        height=900,
        width=1200,
    )

    # Hide redundant x axis labels.
    fig.update_xaxes(
        matches=None, tickangle=-35, title_font_family="Helvetica", tickfont_family="Helvetica", tickfont_size=28
    )
    fig.update_yaxes(title_font_family="Helvetica", tickfont_family="Helvetica", title_font_size=36)
    fig.for_each_xaxis(lambda x: x.update(title=""))
    fig.for_each_annotation(lambda a: a.update(text=""))
    fig.update_traces(
        marker=dict(
            size=14,
        ),
        error_y=dict(thickness=5),
    )

    fig = fig.update_layout(legend=dict(title_font_family="Helvetica", font=dict(size=28)))

    fig = fig.update_layout(
        scattermode="group",
        font_color="black",
        title_font_family="Helvetica",
        font_size=24,
        title_x=0.47,
    )

    plot_output_path = "plot.png"
    fig.write_image(plot_output_path, scale=3)
    with open(plot_output_path, "rb") as img_file:
        image_widget = widgets.Image(value=img_file.read(), format="png", width=700, height=1200)
        display(image_widget)

##### Interative Visualization

In [ ]:
button = widgets.Button(description="Visualize")
button.on_click(visualize_fn)
visualize_fn(button)

Dropdown(description='Model:', options=('(all)', 'LLaMA-7B', 'Llama2-7B', 'OPT-6.7B'), value='(all)')

Dropdown(description='Category:', index=2, options=('grouped_age', 'grouped_country_by_gdp_ppp_quantile', 'gro…

Dropdown(description='Metric:', index=5, options=('accuracy', 'negative_FPR', 'negative_TPR', 'neutral_FPR', '…

Dropdown(description='Dataset:', index=3, options=('(all)', 'SST5', 'SemEval', 'ZeroShot'), value='ZeroShot')

Button(description='Visualize', style=ButtonStyle())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x0e\x10\x00\x00\n\x8c\x08\x06\x00\x00\x00\x83Q]\x11\…

### Compare between models and datasets on the same protected group 
E.g. "young" people of the "age" category.


##### Interative Visualization Utilities

In [ ]:
unique_groups = stats_table["group"].unique()
model_selector_2 = widgets.Dropdown(options=models, value=models[0], description="Model:")
group_selector_2 = widgets.Dropdown(options=unique_groups, value=unique_groups[0], description="Group:")
metric_selector_2 = widgets.Dropdown(options=metrics, value=metrics[0], description="Metric:")

In [ ]:
def visualize_fn_2(button: Any) -> None:
    clear_output()
    display(model_selector_2)
    display(group_selector_2)
    display(metric_selector_2)
    display(button)

    model = model_selector_2.value
    group: str = group_selector_2.value  # type: ignore
    metric: str = metric_selector_2.value  # type: ignore

    gap_metric = metric + "_gap"

    if model == "(all)":
        filtered_table = stats_table[(stats_table["dataset"] != "")]
    else:
        filtered_table = stats_table[(stats_table["model"] == model) & (stats_table["dataset"] != "")]

    group_filtered_table = filtered_table[filtered_table["group"] == group]

    if len(group_filtered_table) == 0:
        print("None of the runs in the table matched the selections.")
        return

    title = f"Gap for {group.title()} "

    fig = px.scatter(
        group_filtered_table,
        x="dataset",
        facet_col="dataset",
        y=gap_metric,
        color="model",
        error_y=gap_metric + "_width",
        labels={
            "group": "Protected Group",
            gap_metric: format_metric_name(gap_metric),
            "model": "LM",
            **{dataset: format_metric_name(dataset) for dataset in filtered_table["dataset"].unique()},
        },
        title=title,
        height=700,
        width=1200,
    )

    # Hide redundant x axis labels.
    fig.update_xaxes(
        matches=None, tickangle=-35, title_font_family="Helvetica", tickfont_family="Helvetica", tickfont_size=28
    )
    fig.update_yaxes(title_font_family="Helvetica", tickfont_family="Helvetica", title_font_size=36)
    fig.for_each_xaxis(lambda x: x.update(title=""))
    fig.for_each_annotation(lambda a: a.update(text=""))
    fig.update_traces(
        marker=dict(
            size=14,
        ),
        error_y=dict(thickness=5),
    )

    fig = fig.update_layout(legend=dict(title_font_family="Helvetica", font=dict(size=28)))

    fig = fig.update_layout(
        scattermode="group",
        font_color="black",
        title_font_family="Helvetica",
        font_size=24,
        title_x=0.47,
    )

    plot_output_path = "plot.png"
    fig.write_image(plot_output_path, scale=5)
    with open(plot_output_path, "rb") as img_file:
        image_widget = widgets.Image(value=img_file.read(), format="png", width=700, height=1200)
        display(image_widget)

##### Interative Visualization

In [ ]:
button_2 = widgets.Button(description="Visualize")
button_2.on_click(visualize_fn_2)
visualize_fn_2(button_2)

Dropdown(description='Model:', options=('(all)', 'LLaMA-7B', 'Llama2-7B', 'OPT-6.7B'), value='(all)')

Dropdown(description='Group:', options=('adult', 'old', 'young', '1q', '2q', '3q', '4q', '5q', '6q', 'chronic_…

Dropdown(description='Metric:', options=('accuracy', 'negative_FPR', 'negative_TPR', 'neutral_FPR', 'neutral_T…

Button(description='Visualize', style=ButtonStyle())

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x17p\x00\x00\r\xac\x08\x06\x00\x00\x00E\x06\x98\x97\…